In [ ]:
#black scholes classes

In [6]:
#initialize libraries
import numpy as np
import pandas as pd
import math
from scipy.stats import norm

In [7]:
#Black Scholes Pricing Model

#S = asset price
#K = strike price
#T = expiration time (yrs)
#r = risk free rate (annualized)
#sig = volatility

# initialize the parent class
class BlackScholes:
    def __init__(self,S,K,T,r,sig):
        self.S = S
        self.K = K
        self.T = T
        self.r = r
        self.sig = sig
        self.d1 = self.calcD1Term()
        self.d2 = self.calcD2Term(self.d1)
    
    #temporary term used in calculation
    def calcD1Term(self):
        d1 = (np.log(self.S/self.K) + (self.r + (self.sig**2 / 2))*self.T)/(self.sig*math.sqrt(self.T))
        return d1
    
    #temporary term used in calculation, based on d1
    def calcD2Term(self,d1):
        d2 = d1 - self.sig*math.sqrt(self.T)
        return d2
    
    #creating functions for normal pdf and cdf in case we want to edit distribution
    def calcCDF(self,dTerm): 
        return norm.cdf(dTerm)

    def calcPDF(self,dTerm): 
        return norm.pdf(dTerm)
    
    # greek calculations that are the same for both put/calls below

    def calcGamma(self):
        return self.calcPDF(self.d1) / (self.S * self.sig * math.sqrt(self.T))

    def calcVega(self):
        return self.S * self.calcPDF(self.d1) * math.sqrt(self.T)
    
    #print function

    def print(self):
        print("The option has the following outputs: ")
        print("Price: ${:.4f}".format(self.calcPrice()))
        print("Delta: ${:.4f}".format(self.calcDelta()))
        print("Gamma: ${:.4f}".format(self.calcGamma()))
        print("Vega: ${:.4f}".format(self.calcVega()))
        print("Theta: ${:.4f}".format(self.calcTheta()))
        print("Rho: ${:.4f}".format(self.calcRho()))
        print()


In [8]:
#child class for call options
class EuropeanCall(BlackScholes):
    def __init__(self,S,K,T,r,sig):
        super().__init__(S,K,T,r,sig)


    def calcPrice(self):
        return self.S * self.calcCDF(self.d1) - self.K *  (math.exp(-self.r * self.T)) * self.calcCDF(self.d2)

    def calcDelta(self):
        return self.calcCDF(self.d1)
    
    def calcTheta(self):
        return - (self.S * self.calcPDF(self.d1)*self.sig)/(2*math.sqrt(self.T)) - (self.r * self.K * math.exp(-self.r * self.T) * self.calcCDF(self.d2))
    
    def calcRho(self):
        return self.K * self.T * math.exp(-self.r * self.T) * self.calcCDF(self.d2)


#child class for put options
class EuropeanPut(BlackScholes):
    def __init__(self,S,K,T,r,sig):
        super().__init__(S,K,T,r,sig)
        self.d1 = self.calcD1Term()
        self.d2 = self.calcD2Term(self.d1)

    def calcPrice(self):
        return self.K *  (math.exp(-self.r * self.T)) * self.calcCDF(-self.d2) - self.S * self.calcCDF(-self.d1)
    
    def calcDelta(self):
        return self.calcCDF(self.d1) - 1
    
    def calcTheta(self):
        return - (self.S * self.calcPDF(self.d1) * self.sig) / (2 * math.sqrt(self.T))+ (self.r * self.K * math.exp(-self.r * self.T) * self.calcCDF(-self.d2))
 
    def calcRho(self):
        return -self.K * self.T * math.exp(-self.r * self.T) * self.calcCDF(-self.d2)
